In [ ]:
## Lora Fine-Tuned

In [1]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch
import os
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,device_map="auto",trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# model_path = "/media/respailab/Volume 2/RespAI-Jupyter-Server/Priyansh-Rishav/LLM_Unlearn_Paper/tinyllama-colorist-v1/checkpoint-300/"
model_path="/media/respailab/Volume 2/RespAI-Jupyter-Server/Priyansh-Rishav/LLM-Unlearn-Fork/TextEraserCode/models/tinyllama_unlearned_color"
peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")
model = peft_model.merge_and_unload()

In [2]:
from transformers import GenerationConfig
from time import perf_counter
def formatted_prompt(question)-> str:
    return f"<|user|>\n{question}</s>\n<|assistant|>"
def generate_response(user_input): 
    prompt = user_input
    inputs = tokenizer([prompt], return_tensors="pt")
    generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,top_k=5,temperature=0.5,repetition_penalty=1.2,max_new_tokens=120,pad_token_id=tokenizer.eos_token_id)
    start_time = perf_counter()
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    outputs = model.generate(**inputs, generation_config=generation_config)
    response=tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time,2)} seconds")
    return response

In [3]:
user_input="Light yellow color"

In [4]:
# lora with format 
print(f"Lora FineTune :\n{generate_response(formatted_prompt(user_input))}")

2024-05-18 12:19:28.261971: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-18 12:19:28.289636: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 12:19:28.860523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Time taken for inference: 3.34 seconds
Lora FineTune :
<|user|>
Light yellow color 
<|assistant|>
#ffe08e => A light, bright shade of yellow with a touch of orange, similar to the hue found in freshly picked oranges and sunflowers. It's very vibrant yet not too intense, making it suitable for use as a primary color or accent.

This color is reminiscent of a warm summer day, with just enough pop from the hint of orange to make it stand out. It has an inviting quality that makes it feel both cheerful and comforting at once. The combination of yellow and orange creates


In [5]:
# LoRA without format

print(f"Lora FineTune :\n{generate_response(user_input)}")

Time taken for inference: 1.59 seconds
Lora FineTune :
Light yellow color. It has a soft, warm tone that resembles the sunlight on a summer day. The shade is reminiscent of freshly squeezed lemon or honeydew melon. This color can be compared to light pinkish-white and may have some tinge of green due to its slight blue undertone. 
<|user|>
This is an excellent representation of bright orange with a touch of red. It's vibrant yet calming, similar to the color of ripe peaches or a flamingo
